# Description
This example is a simple demonstration of how to run a Sesam Core fatigue hotspot example using OneWorkflow locally and in the cloud.

In [29]:

from oneWorkflowToolBox import *
# local workspace, all results will be put here after local or cloud runs
# location of common files for all analysis, has to be below workspacePath and in the folder names CommonFiles
workspacePath = r'C:\Users\kblu\source\repos\improveflowH4-workflow\SesamCoreExample\Workspace'
workspaceId = "SesamCoreExample"
cloud_run = False
#If running locally the code below will also start the local workflow host.
workflow_client = one_workflow_client(workspaceId, workspacePath, cloud_run, tmp=r"c:\OneWorkflowTemp", platform=Platform.Windows)
if (cloud_run):
    workflow_client.login()
workflow_client.upload_common_files(FileOptions(max_size_bytes=524_000,patterns=["**/*.*"], overwrite=True))

The temporary blob storage directory is: c:\OneWorkflowTemp\oc_eun5q7px_blob
The temporary jobs root directory is: c:\OneWorkflowTemp\oc_jb05ofw5_jobs
c:\OneWorkflowTemp
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SesamCoreExample\Workspace\CommonFiles\a.txt to c:\OneWorkflowTemp\oc_eun5q7px_blob\SesamCoreExample\CommonFiles\a.txt
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SesamCoreExample\Workspace\CommonFiles\copyFiles.py to c:\OneWorkflowTemp\oc_eun5q7px_blob\SesamCoreExample\CommonFiles\copyFiles.py
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SesamCoreExample\Workspace\CommonFiles\copyFiles2.py to c:\OneWorkflowTemp\oc_eun5q7px_blob\SesamCoreExample\CommonFiles\copyFiles2.py
Uploading C:\Users\kblu\source\repos\improveflowH4-workflow\SesamCoreExample\Workspace\CommonFiles\Specimen1_HotSpotPlate_SCORE-HOTSPOT.JNL to c:\OneWorkflowTemp\oc_eun5q7px_blob\SesamCoreExample\CommonFiles\Specimen1_HotSpotPlate_SCORE-HOTSPOT.JNL
Uploading C:\U

True

In [30]:
import pandas as pd
import os
from dnv.oneworkflow import  ParallelWork
from oneWorkflowToolBox import run_workflow_async
from dnv.onecompute.flowmodel import WorkUnit
from dnv.sesam.sesam_core_command import *
from dnv.oneworkflow import PythonCommand, CompositeExecutableCommand
import json
parallel_work = ParallelWork()
for index in [1,2]: # iterating over two simple cases, they now will do the same analysis
    loadcase_folder_name = f"LoadCase{index}"
    result_folder_lc = os.path.join(workflow_client.results_directory, loadcase_folder_name)
    python_copy_command = PythonCommand(
        directory=workflow_client.common_directory,
        filename="copyfiles.py",
        working_dir=result_folder_lc)
    core_command = SesamCoreCommand(command = "fatigue",input_file_name= "Specimen1_input.json", options = "-hs")
    cmd = CompositeExecutableCommand([python_copy_command, core_command], result_folder_lc)
    work_unit = (WorkUnit(cmd, loadcase_folder_name)
                 .input_directory(result_folder_lc)
                 .output_directory(result_folder_lc, include_files=["**/*.txt",  "**/*.MLG", "**/*.lis", "**/*.sin"]))
    parallel_work.WorkItems.append(work_unit)
downloadOptions = FileOptions(max_size_bytes=11124_000,patterns=["**/*.txt", "**/*.lis", "**/*.MLG"])
job = workflow_client.create_job(parallel_work)
job_json = json.dumps(job, default=lambda o: o.encode(), indent=4)
await run_workflow_async(job, workflow_client, downloadOptions)


The work item 5d7b03bd-2750-4ab6-847f-09534cf3b66d message is 'Created'
The work item LoadCase1 message is 'Created'
The work item LoadCase2 message is 'Created'
The progress of the job is '0%'. The message is ''
The work item LoadCase1 message is ''
The work item LoadCase2 message is ''
The status of work item '5d7b03bd-2750-4ab6-847f-09534cf3b66d' is 'Completed'
The status of work item 'LoadCase1' is 'Completed'
The status of work item 'LoadCase2' is 'Completed'
